# Structural comparison of manipulated O1v5-SONATA connectomes
This is an auxiliary notebook for analysing manipulated O1v5-SONATA connectomes, which

 - sets up structural comparator
 - runs structural comparisom

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.10.dev1 or later; with v4 config format and `parallel-manipulator` entry point)

ℹ️ Related ticket: [[ACCS-19](https://bbpteam.epfl.ch/project/issues/browse/ACCS-19)] "Rewiring V5 to test reliability results"


In [1]:
# Initialization

""" Global imports """
import json
import os

""" Local imports """
from connectome_manipulator.connectome_comparison import structural_comparator

In [2]:
def default_structcomp_config(output_path, base_name, circuit_specs):
    """Generates a default structural comparator config dict w/o any specific comparison."""
    assert isinstance(circuit_specs, list) and len(circuit_specs) == 2, 'ERROR: Two circuits required for comparison!'

    structcomp_config = {}
    structcomp_config['working_dir'] = os.path.join(output_path, base_name)
    structcomp_config['out_dir'] = structcomp_config['working_dir']
    structcomp_config['circuits'] = {str(i): {'circuit_config': circuit_specs[i]['config'],
                                              'circuit_name': circuit_specs[i]['name']} for i in range(2)}
    structcomp_config['plot_types'] = []

    return structcomp_config

def upper_first(name):
    return name[0].upper() + name[1:]

def add_ConnProb_to_config(structcomp_config, hyper_column, group_by, syn_class='EXC', skip_empty_groups=False):
    """Adds "connection probability per layer" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    if syn_class is None:
        cl_name = ''
        cl_sel = {}
    else:
        assert syn_class in ['EXC', 'INH'], 'ERROR: Synapse class error!'
        cl_name = syn_class[0].upper() * 2
        cl_sel = {'synapse_class': syn_class}  # Same synapse class for pre/post selection
    if hyper_column is None:
        assert group_by in ['layer', 'mtype', 'hypercolumn'], f'ERROR: Grouping by "{group_by}" not supported!'
        if len(cl_name) > 0:
            cl_name = '_' + cl_name
        name = f'ConnPer{upper_first(group_by)}{cl_name}'
        mc_sel = {}
    else:
        assert 0 <= hyper_column <= 6, 'ERROR: Hyper column out of range!'
        assert group_by in ['layer', 'mtype'], f'ERROR: Grouping by "{group_by}" not supported!'
        name = f'ConnPer{upper_first(group_by)}_mc{hyper_column}{cl_name}'
        mc_sel = {'hypercolumn': hyper_column}
    comp_dict = {'name': name,
                 'fct': {'source': 'connectivity',
                         'kwargs': {'group_by': group_by,
                                    'skip_empty_groups': skip_empty_groups,
                                    'sel_src': {**mc_sel, **cl_sel},
                                    'sel_dest': {**mc_sel, **cl_sel}}},
                         'res_sel': ['nsyn_conn', 'conn_prob'],
                         'range_prctile': 100,
                         'fig_size': (11, 3),
                         'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def add_MeanProps_to_config(structcomp_config, hyper_column, props_list, group_by, skip_empty_groups=False):
    """Adds "connection properties (mean) per layer" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    assert 0 <= hyper_column <= 6, 'ERROR: Hyper column out of range!'
    assert isinstance(props_list, list) and len(props_list) > 0, "ERROR: Non-empty list of properties required!"
    assert group_by in ['layer', 'mtype'], f'ERROR: Grouping by "{group_by}" not supported!'
    comp_dict = {'name': f'PropsPer{upper_first(group_by)}_mc{hyper_column}EE',
                         'fct': {'source': 'properties',
                                 'kwargs': {'group_by': group_by,
                                            'skip_empty_groups': skip_empty_groups,
                                            'sel_src': {'hypercolumn': hyper_column, 'synapse_class': 'EXC'},
                                            'sel_dest': {'hypercolumn': hyper_column, 'synapse_class': 'EXC'},
                                            'fct': 'np.mean'}},
                                 'res_sel': props_list,
                                 'range_prctile': 100,
                                 'fig_size': (11, 3),
                                 'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def add_AdjMat_to_config(structcomp_config, hyper_column):
    """Adds "adjacency matrix" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    assert 0 <= hyper_column <= 6, 'ERROR: Hyper column out of range!'
    comp_dict = {'name': f'Adjacency_mc{hyper_column}EE',
                         'fct': {'source': 'adjacency',
                                 'kwargs': {'sel_src': {'hypercolumn': hyper_column, 'synapse_class': 'EXC'},
                                            'sel_dest': {'hypercolumn': hyper_column, 'synapse_class': 'EXC'}}},
                                 'res_sel': ['adj', 'adj_cnt'],
                                 'range_prctile': 95,
                                 'fig_size': (11, 3),
                                 'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def export_structcomp_config(structcomp_config, config_path):
    """Writes structural comparator config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    base_name = os.path.split(structcomp_config['working_dir'])[-1]
    fn = f'structcomp_config__{base_name}__{structcomp_config["circuits"]["0"]["circuit_name"]}_vs_{structcomp_config["circuits"]["1"]["circuit_name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(structcomp_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

def print_launch_cmd(config_path, config_fn, force_recomp=False):
    if force_recomp:
        run_cmd = f"sbatch run_struct_comparison.sh {config_fn} --force-recomp-circ1 --force-recomp-circ2"
    else:
        run_cmd = f"sbatch run_struct_comparison.sh {config_fn}"
    print(f"# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:")
    print(f"cd {config_path}")
    print(run_cmd)

def load_config_from_file(path, fn):
    config_file = os.path.join(path, fn)
    with open(config_file, 'r') as f:
        structcomp_config = json.load(f)
    return structcomp_config

### Selection of circuits to compare

In [3]:
# Circuit base name
circuit_name_base = 'O1v5-SONATA'

# Original circuit
circuit_name_orig = 'Orig'
circuit_config_orig = f'/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/{circuit_name_base}/sonata/circuit_config_tmp.json' # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]
circuit_spec_orig = {'name': circuit_name_orig, 'config': circuit_config_orig}

# # Manipulated circuits - Enhanced structure
# enhance_N = [100, 200, 300, 400, 500, 670]
# circuit_names_manip = [f'Enhanced{_n}K' for _n in enhance_N]
# circuit_configs_manip = [f'/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/{circuit_name_base}__ConnRewireEnhanced{_n}Kmc2EE/circuit_config.json' for _n in enhance_N]
# circuit_specs_manip = [{'name': nm, 'config': cfg} for nm, cfg in zip(circuit_names_manip, circuit_configs_manip)]

# Manipulated circuits - Added RCs
# circuit_names_manip = ['struct0_2x', 'struct0_4x', 'control0_2x', 'control0_4x']  # (Part I)
circuit_names_manip = ['struct0_8x', 'struct0_16x', 'control0_8x', 'control0_16x']  # (Part II)
circuit_configs_manip = [f'/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/{circuit_name_base}__ConnAddRC_mc2EE_{_nm}/circuit_config.json' for _nm in circuit_names_manip]
circuit_specs_manip = [{'name': nm, 'config': cfg} for nm, cfg in zip(circuit_names_manip, circuit_configs_manip)]

# # Manipulated circuits - Removed RCs
# circuit_names_manip = ['StructDim56-0', 'StructDim56_456-0', 'StructDim456-0', 'Unstruct-0', 'Unstruct-1', 'Unstruct-2', 'Unstruct-3']
# circuit_configs_manip = [f'/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/{circuit_name_base}__RecipRemoval_{_nm}/sonata/circuit_config_RecipRemoval_{_nm}.json' for _nm in circuit_names_manip]
# circuit_specs_manip = [{'name': nm, 'config': cfg} for nm, cfg in zip(circuit_names_manip, circuit_configs_manip)]

### Configuration of structural comparison

In [4]:
# # Output paths - Enhanced structure
# output_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/struct_comparison'
# config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/enhanced_connectivity/configs']  # Write to local and proj9 folder

# Output paths - Added RCs
output_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/struct_comparison'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs']  # Write to local and proj9 folder

# # Output paths - Removed RCs
# output_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_removal/struct_comparison'
# config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_removal/configs']  # Write to local and proj9 folder

# Selection of column
hyper_column = 2  # 2...central column

# Selection of properties to compare
props_list = ['conductance', 'decay_time', 'delay', 'depression_time', 'facilitation_time', 'syn_type_id', 'u_syn']

In [5]:
# Structural comparison of original vs. simplified connectomes
for idx in range(len(circuit_names_manip)):
    structcomp_config = default_structcomp_config(output_path, circuit_name_base, [circuit_spec_orig, circuit_specs_manip[idx]])
    add_ConnProb_to_config(structcomp_config, hyper_column, group_by='layer', skip_empty_groups=False)
    add_ConnProb_to_config(structcomp_config, hyper_column, group_by='mtype', skip_empty_groups=True)
    add_MeanProps_to_config(structcomp_config, hyper_column, props_list=props_list, group_by='layer', skip_empty_groups=False)
    add_MeanProps_to_config(structcomp_config, hyper_column, props_list=props_list, group_by='mtype', skip_empty_groups=True)
    add_AdjMat_to_config(structcomp_config, hyper_column)
    export_structcomp_config(structcomp_config, config_paths)

Config file structcomp_config__O1v5-SONATA__Orig_vs_struct0_8x.json written to ../configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_struct0_8x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_struct0_16x.json written to ../configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_struct0_16x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_control0_8x.json written to ../configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_control0_8x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_control0_16x.json written to ../configs
Config file structcomp_config__O1v5-SONATA__Orig_vs_control0_16x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/conf

### Run structural comparison

Launch using `sbatch` first, in order to extract/compute the results:

In [6]:
for _idx, _nm in enumerate(circuit_names_manip):
    print_launch_cmd(config_paths[1], f'structcomp_config__{circuit_name_base}__Orig_vs_{_nm}.json', force_recomp=False)
    print()
    if _idx == 0:
        print('WAIT UNTIL "ORIG" RESULTS ARE READY BEFORE LAUNCHING THE OTHERS!')
        print()

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_struct_comparison.sh structcomp_config__O1v5-SONATA__Orig_vs_struct0_8x.json

WAIT UNTIL "ORIG" RESULTS ARE READY BEFORE LAUNCHING THE OTHERS!

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_struct_comparison.sh structcomp_config__O1v5-SONATA__Orig_vs_struct0_16x.json

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_struct_comparison.sh structcomp_config__O1v5-SONATA__Orig_vs_control0_8x.json

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_struct_comparison.sh structcomp_config__O1v5-SONATA__Orig_vs_control0_16x.json



If needed, re-run directly within notebook w/o re-computing, to visualize figures within notebook:

In [1]:
# # Load config & re-run comparison for plotting
# for _nm in circuit_names_manip:
#     structcomp_config = load_config_from_file(config_paths[-1], f'structcomp_config__{circuit_name_base}__Orig_vs_{_nm}.json')
#     structural_comparator.main(structcomp_config, show_fig=True, force_recomp=[False, False])

### [Optional] Copy results figures in single folder for simple overview

In [15]:
import shutil

base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny'
res_paths = ['simplified_connectome_models/struct_comparison/O1v5-SONATA/Orig_vs_Order-1',
             'simplified_connectome_models/struct_comparison/O1v5-SONATA/Orig_vs_Order-2',
             'simplified_connectome_models/struct_comparison/O1v5-SONATA/Orig_vs_Order-3',
             'simplified_connectome_models/struct_comparison/O1v5-SONATA/Orig_vs_Order-4',
             'simplified_connectome_models/struct_comparison/O1v5-SONATA/Orig_vs_Order-5',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced100K',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced200K',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced300K',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced400K',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced500K',
             'enhanced_connectivity/struct_comparison/O1v5-SONATA/Orig_vs_Enhanced670K',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_StructDim56-0',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_StructDim56_456-0',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_StructDim456-0',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_Unstruct-0',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_Unstruct-1',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_Unstruct-2',
             'reciprocal_removal/struct_comparison/O1v5-SONATA/Orig_vs_Unstruct-3',
             'reciprocal_addition/struct_comparison/O1v5-SONATA/Orig_vs_struct0_2x',
             'reciprocal_addition/struct_comparison/O1v5-SONATA/Orig_vs_struct0_4x',
             'reciprocal_addition/struct_comparison/O1v5-SONATA/Orig_vs_control0_2x',
             'reciprocal_addition/struct_comparison/O1v5-SONATA/Orig_vs_control0_4x']
res_files = ['struct_comp-Adjacency_mc2EE-adj.png',
             'struct_comp-ConnPerLayer_mc2EE-conn_prob.png',
             'struct_comp-ConnPerMtype_mc2EE-conn_prob.png']

save_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs'

In [19]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

for _path in res_paths:
    for _fn in res_files:
        res_file = os.path.join(base_path, _path, _fn)
        assert os.path.exists(res_file), f'ERROR: File "{res_file}" does not exist!'

        # save_fn = f'{_path.split(os.path.sep)[0]}__{_path.split(os.path.sep)[-1]}__{_fn}'  # Sorted by manipulation
        save_fn = f'{os.path.splitext(_fn)[0]}__{_path.split(os.path.sep)[0]}__{_path.split(os.path.sep)[-1]}{os.path.splitext(_fn)[-1]}'  # Sorted by res file
        save_file = os.path.join(save_path, save_fn)
        shutil.copyfile(res_file, save_file)
        print(f'INFO: File copied to {save_file}')

INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs/struct_comp-Adjacency_mc2EE-adj__simplified_connectome_models__Orig_vs_Order-1.png
INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs/struct_comp-ConnPerLayer_mc2EE-conn_prob__simplified_connectome_models__Orig_vs_Order-1.png
INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs/struct_comp-ConnPerMtype_mc2EE-conn_prob__simplified_connectome_models__Orig_vs_Order-1.png
INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs/struct_comp-Adjacency_mc2EE-adj__simplified_connectome_models__Orig_vs_Order-2.png
INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struct_comparison_figs/struct_comp-ConnPerLayer_mc2EE-conn_prob__simplified_connectome_models__Orig_vs_Order-2.png
INFO: File copied to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/struc